# Project : Social Network Analysis

Author: Geethasree Madiraju Nagaraju  
Date: 05/21/2023


# Project Background
Online social networks have become ubiquitous in our society. Although the first such network is
hard to determine, it is commonly believed to have been the website SixDegrees which was stared
in 1997 and lasted approximately 4 years. From this website the social media craze was born,
with Friendster, Myspace, LinkedIn, Facebook, Twitter, and many others spawning and growing in
use and popularity since then, connecting people from all over the world in ways never previously
imagined.
Social networks are not just used for connecting people —they have become one of the main mechanisms for companies to reach customers through targeted marketing, recruiting, increasing company
credibility, increasing the quality of feedback on products on services, among many others. These
tasks, and many others, rely on Social Network Analysis (SNA), which, according to Wikipedia1
,
can be defined as ‘the process of investigating social structures through the use of networks and
network theories.’
In this project, you will be given two social network networks, one small and one large, and be asked
to build spreadsheet models to solve some problems some typically SNA problem. The files are in
Excel file SocialNetworks.xlsx in sheets Small and Large. The Small sheet contains a social
network with 25 individuals and the Large sheet contains a social network with 50 individuals. For
each individuals, in sequence, the links that they have in the network are given. For example, in
the Large network, individual 9 is connected with individuals 7,8,26, and 29. You are to answer
each of the following questions for each of the networks.


In [ ]:
import sys
import os
 
if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from pyomo.environ import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/Colab Notebooks/SocialNetworks.xlsx"

In [ ]:
# Read Graph data from xlsx into Adjacency Matrix
small_network = pd.read_excel(data_path,sheet_name='Small',header=None)
large_network = pd.read_excel(data_path,sheet_name='Large',header=None)
n_individuals_small = small_network.shape[0]
n_individuals_large = large_network.shape[0]
adj_mat_small = np.zeros((n_individuals_small,n_individuals_small))
adj_mat_large = np.zeros((n_individuals_large,n_individuals_large))
print(small_network.shape)
print(large_network.shape)

In [ ]:
# Read into Adjacency Matrix - small network
for i in range(n_individuals_small):
  for j in [2,3,4,5,6]:
    conn = small_network.loc[i][j]
    if math.isnan(conn):
      break
    else:
      adj_mat_small[i,int(conn-1)] = 1

# Read into Adjacency Matrix - large network
for i in range(n_individuals_large):
  for j in [2,3,4,5,6,7,8,9]:
    conn = large_network.loc[i][j]
    if math.isnan(conn):
      break
    else:
      adj_mat_large[i,int(conn-1)] = 1


# Problem 1.a
(a) One of the main metrics used to describe a social network is the density — that is, the
number of links in the network over the number of possible links. What is the density
of each network?

* Density of the network = number of links in the network/number of all possible links  
* number of possible all links among $n$ nodes = $n.(n-1)/2 $ for an undirected graph

In [ ]:
# Density of the small network - Since adjacency matrix is symmetric, all the connections will be double counted
density = np.sum(adj_mat_small)/((n_individuals_small**2) - n_individuals_small)
print("Density of the small network: ",density)

# Density of the large network
density = np.sum(adj_mat_large)/((n_individuals_large**2) - n_individuals_large)
print("Density of the large network: ",density)

# Problem 1.b - Shortest Path
(b) For job seekers, LinkedIn has become an invaluable resource for finding connections at
companies they are interested in obtaining a job offer from. The website provides a
platform for people to connect with professional contacts. Suppose the networks in the
examples are a part of the LinkedIn network and you, the person with index 1, are
seeking a job at a company where individual 16 works. What is the fewest number
of connections between you and individual 16? What is the path from you to that
individuals, in each of the networks?

* Fewest number of connections between two nodes in a network = number of connections in the shortest path between source node and destination node = length of shortest path - 1.

In [ ]:
# declaring source and destination nodes between which shortest path need to be found out(based on zero indexing)
source_node = 0
destination_node = 15

### Small Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables
model.x = Var(range(n_individuals_small),range(n_individuals_small),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    objective_expr += model.x[i,j]*adj_mat_small[i,j]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# There cannot exist a path between nodes that are not connected
model.does_not_exist_constraints = ConstraintList()
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    if adj_mat_small[i,j] < 0.001:
      model.does_not_exist_constraints.add(model.x[i,j] == 0)

#Node flow constraints
#for all the intermediate nodes : number of outgoing edges - number of incoming edges = 0
# for source node : number of outgoing edges - number of incoming edges = 1
# for destination node : number of outgoing edges - number of incoming edges = -1
model.node_flow = ConstraintList()
for i in range(n_individuals_small):
  node_incoming_total = 0
  node_outgoing_total = 0
  for j in range(n_individuals_small):
    node_outgoing_total += model.x[i,j]
    node_incoming_total += model.x[j,i]
  if i==source_node:
    model.node_flow.add(node_outgoing_total-node_incoming_total == 1)
  elif i==destination_node:
    model.node_flow.add(node_outgoing_total-node_incoming_total == -1)
  else:
    model.node_flow.add(node_outgoing_total-node_incoming_total == 0)


model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the shortest path from source to destination
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    if model.x[i,j]() > 0.5:
      print(i+1, "-->", j+1)
print("Shortest Path length: ",model.objective_expr())
print("Fewest number of connections between source and destination ",model.objective_expr()-1)

### Large Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables
model.x = Var(range(n_individuals_large),range(n_individuals_large),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    objective_expr += model.x[i,j]*adj_mat_large[i,j]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# There cannot exist a path between nodes that are not connected
model.does_not_exist_constraints = ConstraintList()
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    if adj_mat_large[i,j] < 0.001:
      model.does_not_exist_constraints.add(model.x[i,j] == 0)

#Node flow constraints
#for all the intermediate nodes : number of outgoing edges - number of incoming edges = 0
# for source node : number of outgoing edges - number of incoming edges = 1
# for destination node : number of outgoing edges - number of incoming edges = -1
model.node_flow = ConstraintList()
for i in range(n_individuals_large):
  node_incoming_total = 0
  node_outgoing_total = 0
  for j in range(n_individuals_large):
    node_outgoing_total += model.x[i,j]
    node_incoming_total += model.x[j,i]
  if i==source_node:
    model.node_flow.add(node_outgoing_total-node_incoming_total == 1)
  elif i==destination_node:
    model.node_flow.add(node_outgoing_total-node_incoming_total == -1)
  else:
    model.node_flow.add(node_outgoing_total-node_incoming_total == 0)


model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the shortest path from source to destination
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    if model.x[i,j]() > 0.5:
      print(i+1, "-->", j+1)
print("Shortest Path length: ",model.objective_expr())
print("Fewest number of connections between source and destination ",model.objective_expr()-1)

# Problem 1.c - Diversity
Diversity in a social network gives a measure of how different people in a social network
are, and hence a measure of the potential for new, interesting connections. Although there are several ways of measuring diversity, in the problem, you are to determine, for both networks, what is the size of the largest group of people, no two of which are connected to each other.
* Size of largest group of people no two of which are connected to each other



### Small Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is in the diverse group or not
model.x = Var(range(n_individuals_small),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_small):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Any two nodes should not be directly connected to each other in a diverse group
# if a node is present, all its connected nodes should not be present in the group
model.connected_nodes = ConstraintList()
for i in range(n_individuals_small):
  sum_connected_nodes_expr =0
  total_connected_nodes = 0
  for j in range(n_individuals_small):
    sum_connected_nodes_expr += model.x[j]*adj_mat_small[i][j]
    total_connected_nodes += adj_mat_small[i][j]
  model.connected_nodes.add(sum_connected_nodes_expr <= total_connected_nodes*(1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()#couenne

In [ ]:
#print the members of largest diverse group - Small Network
for i in range(n_individuals_small):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("Size of largest diverse group: ",model.objective_expr())

### Large Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is in the diverse group or not
model.x = Var(range(n_individuals_large),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_large):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Any two nodes should not be directly connected to each other in a diverse group
# if a node is present, all its connected nodes should not be present in the group
model.connected_nodes = ConstraintList()
for i in range(n_individuals_large):
  sum_connected_nodes_expr =0
  total_connected_nodes = 0
  for j in range(n_individuals_large):
    sum_connected_nodes_expr += model.x[j]*adj_mat_large[i][j]
    total_connected_nodes += adj_mat_large[i][j]
  model.connected_nodes.add(sum_connected_nodes_expr <= total_connected_nodes*(1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()#couenne

In [ ]:
#print the members of largest diverse group - Large Network
for i in range(n_individuals_large):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("Size of largest diverse group: ",model.objective_expr())

# Problem 1.d - Cliques
Cliques are also of interest. A clique is a group of people, all of whom know each other.
The larger the cliques in a network, the more interconnected it is. What is the size of the
largest clique in each network? Which individuals are in the largest cliques? Can you
find any other cliques of the same size within the respective networks? Find as many as
you can.
* Size of largest group of people, all of whom know each other

### Small Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is in the clique or not
model.x = Var(range(n_individuals_small),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_small):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Any two nodes should  be directly connected to each other in a clique
# if a node is present, all its non-connected nodes should not be present in the clique
model.non_connected_nodes = ConstraintList()
for i in range(n_individuals_small):
  sum_non_connected_nodes_expr =0
  total_non_connected_nodes = 0
  for j in range(n_individuals_small):
    if j!= i:
      sum_non_connected_nodes_expr += model.x[j]*(1-adj_mat_small[i][j])
      total_non_connected_nodes += (1-adj_mat_small[i][j])
  model.non_connected_nodes.add(sum_non_connected_nodes_expr <= total_non_connected_nodes*(1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the members of largest clique - Small network
for i in range(n_individuals_small):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("size of largest clique: ",model.objective_expr())

### Other cliques of same size within the network

In [ ]:
# Adding constraints to not select the optimal solution selected thus far, but have same optimal value
optimal_value = model.objective_expr()
model.solution_constraint = ConstraintList()
while True:
  sum_solution_expr = 0
  for i in range(n_individuals_small):
    if model.x[i]() > 0.5:
      sum_solution_expr += model.x[i]
    else:
      sum_solution_expr += (1-model.x[i])
  model.solution_constraint.add(sum_solution_expr<=n_individuals_small - 1)
  SolverFactory('cbc').solve(model, tee=False)#.write()
  if (model.objective_expr() < optimal_value-0.5):
    break
  for i in range(n_individuals_small):
    if model.x[i]() > 0.5:
      print(i+1, end="\t")
  print("\n")
  print("size of this clique: ",model.objective_expr())



### Large Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is in the clique or not
model.x = Var(range(n_individuals_large),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_large):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Any two nodes should  be directly connected to each other in a clique
# if a node is present, all its non-connected nodes should not be present in the clique
model.non_connected_nodes = ConstraintList()
for i in range(n_individuals_large):
  sum_non_connected_nodes_expr =0
  total_non_connected_nodes = 0
  for j in range(n_individuals_large):
    if j!= i:
      sum_non_connected_nodes_expr += model.x[j]*(1-adj_mat_large[i][j])
      total_non_connected_nodes += (1-adj_mat_large[i][j])
  model.non_connected_nodes.add(sum_non_connected_nodes_expr <= total_non_connected_nodes*(1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the members of largest clique - Large network
for i in range(n_individuals_large):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("size of largest clique: ",model.objective_expr())

### Other cliques of same size within the network

In [ ]:
# Adding constraints to not select the optimal solution thus far, but have same optimal value
optimal_value = model.objective_expr()
model.solution_constraint = ConstraintList()
while True:
  sum_solution_expr = 0
  for i in range(n_individuals_large):
    if model.x[i]() > 0.5:
      sum_solution_expr += model.x[i]
    else:
      sum_solution_expr += (1-model.x[i])
  model.solution_constraint.add(sum_solution_expr<=n_individuals_large - 1)
  SolverFactory('cbc').solve(model, tee=False)#.write()
  if (model.objective_expr() < optimal_value-0.5):
    break
  for i in range(n_individuals_large):
    if model.x[i]() > 0.5:
      print(i+1, end="\t")
  print("\n")
  print("size of this clique: ",model.objective_expr())



# Problem 2.a - Linkedin Premium free trial
Perhaps the industry that has found the most use of present day SNA is marketing. SNA
gives a way of connecting with customers, and in the following problems you will answer
questions that are of interest to marketers. This will be in the context of LinkedIn, but you
can also think of this any company deciding who to market new products or services to in
order to reach the most number of other people in the network.

LinkedIn is a Freemium service. Membership is free, but there is also a paid membership where you get additional services. In order to promote the paid membership, the
company often releases one-month ‘trials’ to individuals on the site to temp them into
buying the service after the promotion expires. But, it is not only these individuals
that they are tempting — everyone connected to someone with a paid membership (or
with a current trial) sees that their connection has the paid membership, which thereby
encourages them as well to pay for a membership. LinkedIn would like to run a promotion for its paid memberships, and wants to release the fewest number of trials while
ensuring that everyone in the network is connected to at least one person who is given a
free trial. Suppose that in the networks provided, no individual has a pair membership,
yet. What is the fewest number of people that the trial should be given to in order for
every individual to be either given the trial, or be connected directly to someone who
has? Answer this for both networks.

* fewest number of people to be offered a trial such that every individual is connected to atleast one person who is offered a trial

### Small network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is given a free trial or not
model.x = Var(range(n_individuals_small),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_small):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# if a node is not given a free trial, atleast one of its connected nodes should be given a free trial
model.connected_nodes = ConstraintList()
for i in range(n_individuals_small):
  sum_connected_nodes_expr =0
  total_connected_nodes = 0
  for j in range(n_individuals_small):
    sum_connected_nodes_expr += model.x[j]*adj_mat_small[i][j]
    total_connected_nodes += adj_mat_small[i][j]
  model.connected_nodes.add(sum_connected_nodes_expr <= total_connected_nodes*(1-model.x[i]))
  model.connected_nodes.add(sum_connected_nodes_expr >= (1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the users who should be offered a free trial so that everyone is else connected to atleast one user with free trial 
for i in range(n_individuals_small):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("fewest number of people to be offered linkedin free trial ",model.objective_expr())

### Large Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each of the user is given a free trial or not
model.x = Var(range(n_individuals_large),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_individuals_large):
    objective_expr += model.x[i]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# if a node is not given a free trial, atleast one of its connected nodes should be given a free trial
model.connected_nodes = ConstraintList()
for i in range(n_individuals_large):
  sum_connected_nodes_expr =0
  total_connected_nodes = 0
  for j in range(n_individuals_large):
    sum_connected_nodes_expr += model.x[j]*adj_mat_large[i][j]
    total_connected_nodes += adj_mat_large[i][j]
  model.connected_nodes.add(sum_connected_nodes_expr <= total_connected_nodes*(1-model.x[i]))
  model.connected_nodes.add(sum_connected_nodes_expr >= (1-model.x[i]))
  

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print the users who should be offered a free trial so that everyone is else connected to atleast one user with free trial 
for i in range(n_individuals_large):
  if model.x[i]() > 0.5:
    print(i+1, end="\t")
print("\n")
print("fewest number of people to be offered linkedin free trial ",model.objective_expr())

# Problem 2.b
LinkedIn, like any company, has a limited budget and can only release the free trial to
a limited number of individuals. If LinkedIn has only two free trials to release, which
two individuals should they release the free trial to in order to maximize the number
of people that are connected to someone who receives the trail? Answer this for both
networks.
* Given that only two free trials are available, which are two best people to offer, maximize number of people connected to those two people

### Small Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each pair of nodes is chosen to be given a free trial or not
model.x = Var(range(n_individuals_small),range(n_individuals_small),within=Binary)

# Objective Function - total number of distinctly connected nodes to the selected node pair 
#pre-calculate a matrix connectedness, of shape n_individuals_small x n_individuals_small where connectedness[i,j] contains total number of distinctly connected
#nodes to node_i and node_j (excluding  node_i , node_j)
connectedness  = np.zeros((n_individuals_small,n_individuals_small))
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    if j!=i:
      connectedness[i,j] = sum(adj_mat_small[i]) + sum(adj_mat_small[j]) - 2*adj_mat_small[i,j] - np.dot(adj_mat_small[i],adj_mat_small[j])
objective_expr = 0
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    objective_expr += model.x[i,j]*connectedness[i,j]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Only two individuals can be offered free trial
# same individual cannot be offered two free trials
model.budget_constraint =  ConstraintList()
sum_free_trial_users = 0
same_users_expr = 0
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    sum_free_trial_users += model.x[i,j]
  same_users_expr += model.x[i,i]

model.budget_constraint.add(sum_free_trial_users == 1)
model.budget_constraint.add(same_users_expr == 0)


model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print two best users that need to be offered a free trial
for i in range(n_individuals_small):
  for j in range(n_individuals_small):
    if model.x[i,j]() > 0.5:
      print(i+1, "\t", j+1)
print("Number of connections covered by offering these two free trials: ",model.objective_expr())

### Large Network

In [ ]:
#Create a Model
model = ConcreteModel()

#Variables - 0/1 variable indicating whether each pair of nodes is chosen to be given a free trial or not
model.x = Var(range(n_individuals_large),range(n_individuals_large),within=Binary)

# Objective Function - total number of distinctly connected nodes to the selected node pair 
#pre-calculate a matrix connectedness, of shape n_individuals x n_individuals where connectedness[i,j] contains total number of distinctly connected
#nodes to node_i and node_j (excluding  node_i , node_j)
connectedness  = np.zeros((n_individuals_large,n_individuals_large))
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    if j!=i:
      connectedness[i,j] = sum(adj_mat_large[i]) + sum(adj_mat_large[j]) - 2*adj_mat_large[i,j] - np.dot(adj_mat_large[i],adj_mat_large[j])
objective_expr = 0
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    objective_expr += model.x[i,j]*connectedness[i,j]

model.objective_expr = Objective(expr=objective_expr,sense=maximize)

# Constraints
# Only two individuals can be offered free trial
# same individual cannot be offered two free trials
model.budget_constraint =  ConstraintList()
sum_free_trial_users = 0
same_users_expr = 0
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    sum_free_trial_users += model.x[i,j]
  same_users_expr += model.x[i,i]

model.budget_constraint.add(sum_free_trial_users == 1)
model.budget_constraint.add(same_users_expr == 0)


model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print two best users that need to be offered a free trial
for i in range(n_individuals_large):
  for j in range(n_individuals_large):
    if model.x[i,j]() > 0.5:
      print(i+1, "\t", j+1)
print("\n")
print("Number of connections covered by offering these two free trials: ",model.objective_expr())

# Problem 3 - Cluster Analysis

### Small Network

Cluster analysis is one of the most common tasks in statistical data analysis and data mining,
consisting of grouping a set of objects in such a way that the objects within each group (called
a cluster ) are very similar. Using the networks provided, suppose that the links indicate if
two individuals cannot be in the same cluster because they are not compatible (note, this is
a different interpretation of the links than what was assumed previously). Find the minimum
number of clusters necessary to cover each of the individuals so that individuals in each cluster
are all linked to each other.

In [ ]:
#Create a Model
model = ConcreteModel()

# Variables x (n_clusters, n_individuals) - binary variable indicates whether a user is a part of a cluster or not
# y - indicates whether there exists an element in that cluster or not 
n_clusters = n_individuals_small
model.x = Var(range(n_clusters),range(n_individuals_small),within=Binary)
model.y = Var(range(n_clusters),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_clusters):
    objective_expr += model.y[i]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# Each node should be there in exactly one cluster
model.cluster_individual = ConstraintList()

for j in range(n_individuals_small):
  cluster_per_individual_expr = 0
  for i in range(n_clusters):
    cluster_per_individual_expr += model.x[i,j]
  model.cluster_individual.add(cluster_per_individual_expr == 1)

# Each cluster can have only those nodes that are all connected to each other 
# For all the clusters, if a node is present in a cluster, all the incompatible nodes cannot be present in the same cluster
model.connected_cluster = ConstraintList()
for i in range(n_clusters):
  for j in range(n_individuals_small):
    incompatible_nodes = sum([adj_mat_small[j,k]*model.x[i,k] for k in range(n_individuals_small)])
    model.connected_cluster.add(model.x[i,j]<= 1- incompatible_nodes)

#channeling constraints
# y[i] = 1 if there is atleast one element in the cluster,  0 otherwise
model.chanelling_constraint = ConstraintList()
nodes_in_cluster_expr = 0
for i in range(n_clusters):
  for j in range(n_individuals_small):
    nodes_in_cluster_expr += model.x[i,j]
  model.chanelling_constraint.add(nodes_in_cluster_expr>=model.y[i])
  model.chanelling_constraint.add(nodes_in_cluster_expr<=n_individuals_small*model.y[i])

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print cluster number and users assigned to each cluster
for i in range(n_clusters):
  if model.y[i]() > 0.5:
    print (i+1, end=":\t")
    for j in range(n_individuals_small):
      if model.x[i,j]() > 0.5:
        print(j+1, end="\t")
    print("\n")
print("Minimum number of clusters such that every person in the cluster is linked to each other: ",model.objective_expr())

### Large network

In [ ]:
#Create a Model
model = ConcreteModel()

# Variables x (n_clusters, n_individuals) - binary variable indicates whether a user is a part of a cluster or not
# y - indicates whether there exists an element in that cluster or not 
n_clusters = n_individuals_large
model.x = Var(range(n_clusters),range(n_individuals_large),within=Binary)
model.y = Var(range(n_clusters),within=Binary)

# Objective Function
objective_expr = 0
for i in range(n_clusters):
    objective_expr += model.y[i]

model.objective_expr = Objective(expr=objective_expr,sense=minimize)

# Constraints
# Each node should be there in exactly one cluster
model.cluster_individual = ConstraintList()

for j in range(n_individuals_large):
  cluster_per_individual_expr = 0
  for i in range(n_clusters):
    cluster_per_individual_expr += model.x[i,j]
  model.cluster_individual.add(cluster_per_individual_expr == 1)

# Each cluster can have only those nodes that are all connected to each other 
# For all the clusters, if a node is present in a cluster, all the incompatible nodes cannot be present in the same cluster
model.connected_cluster = ConstraintList()
for i in range(n_clusters):
  for j in range(n_individuals_large):
    incompatible_nodes = sum([adj_mat_large[j,k]*model.x[i,k] for k in range(n_individuals_large)])
    model.connected_cluster.add(model.x[i,j]<= 1- incompatible_nodes)

#channeling constraints
# y[i] = 1 if there is atleast one element in the cluster,  0 otherwise
model.chanelling_constraint = ConstraintList()
nodes_in_cluster_expr = 0
for i in range(n_clusters):
  for j in range(n_individuals_large):
    nodes_in_cluster_expr += model.x[i,j]
  model.chanelling_constraint.add(nodes_in_cluster_expr>=model.y[i])
  model.chanelling_constraint.add(nodes_in_cluster_expr<=n_individuals_large*model.y[i])

model.pprint()

SolverFactory('cbc').solve(model, tee=True).write()

In [ ]:
#print cluster number and users assigned to each cluster
for i in range(n_clusters):
  if model.y[i]() > 0.5:
    print (i+1, end=":\t")
    for j in range(n_individuals_large):
      if model.x[i,j]() > 0.5:
        print(j+1, end="\t")
    print("\n")
print("\n")
print("Minimum number of clusters such that every person in the cluster is linked to each other: ",model.objective_expr())